In [1]:
import os

os.environ['R_HOME'] = '/home/snandi/anaconda3/envs/py_311_r43/lib/R'
%load_ext rpy2.ipython

In [2]:
%%R

read_count_dis_co_occu_d21_t21=read.csv('./output/plier_LVS_D21_T2_BMI_htp_for_deseq2.csv', header = TRUE)

# Convert the first column to row names
rownames(read_count_dis_co_occu_d21_t21) <- read_count_dis_co_occu_d21_t21[, ncol(read_count_dis_co_occu_d21_t21)]

# Remove the first column from the data frame
read_count_dis_co_occu_d21_t21 <- read_count_dis_co_occu_d21_t21[, -ncol(read_count_dis_co_occu_d21_t21)]

# Display the first few rows of the data
#head(read_count_dis_co_occu_d21_t21)

In [3]:
%%R

meta_data_dis_co_occu_d21_t21=read.csv('./output/HTP_meta_data_D21_T21_BMI.csv', header = TRUE)

# Convert the first column to row names
rownames(meta_data_dis_co_occu_d21_t21) <- meta_data_dis_co_occu_d21_t21$LabID

# Remove the first column from the data frame
#read_count_dis_co_occu_d21_t21 <- read_count_dis_co_occu_d21_t21[, -1]

# Display the first few rows of the data
head(meta_data_dis_co_occu_d21_t21)

             Sex Karyotype Event_name     LabID Age_at_visit      BMI
HTP0001B2 Female   Control    Visit 3 HTP0001B2         25.8 21.92959
HTP0005A3   Male       T21    Visit 3 HTP0005A3         36.5 24.34652
HTP0012A2 Female       T21    Visit 2 HTP0012A2         44.0 23.14031
HTP0015A4   Male       T21    Visit 4 HTP0015A4         31.2 62.34889
HTP0017A4 Female       T21    Visit 4 HTP0017A4         23.6 24.17873
HTP0022B2 Female   Control    Visit 2 HTP0022B2         43.2 27.95809
          Sample_source Anxiety Any.autoimmune.skin.condition
HTP0001B2         Local       1                             0
HTP0005A3         Local       1                             0
HTP0012A2         Local       0                             1
HTP0015A4         Local       0                             1
HTP0017A4         Local       0                             0
HTP0022B2         Local       1                             0
          Any.congenital.heart.defect Any.hearing.loss.condition
HTP0001B2  

In [5]:
%%R
group=meta_data_dis_co_occu_d21_t21$BMI_dis

In [6]:
%%R
# Load the limma package
library(limma)

# Example expression matrix M (rows are genes, columns are samples)
#M <- matrix(rnorm(1000), nrow=100, ncol=10)  # Simulated data

# Create a design matrix
#group <- factor(c(rep("Control", 5), rep("Treatment", 5)))
design <- model.matrix(~group)

# Fit the linear model
fit <- lmFit(read_count_dis_co_occu_d21_t21, design)

# Examine the fit object
head(fit)

fit <- eBayes(fit)
results <- topTable(fit, number=Inf)


# Create the volcano plot
volcano_data <- data.frame(
    LVs=rownames(results),
    logFC = results$logFC,
    adj_Pval=results$adj.P.Val,
    negLogadj_Pval = -log10(results$adj.P.Val)
)

R[write to console]: Removing intercept from test coefficients



In [7]:
%%R

head(volcano_data)


    LVs      logFC     adj_Pval negLogadj_Pval
1  LV37 -5.8620505 3.330138e-06       5.477538
2  LV43  2.7419379 2.468918e-04       3.607493
3  LV34  3.4855795 2.468918e-04       3.607493
4   LV6 -6.1226084 2.653734e-04       3.576143
5  LV83  1.8089640 1.841066e-03       2.734931
6 LV100  0.9929464 2.852902e-03       2.544713


In [8]:
%%R

fit$gene_names=rownames(fit)

In [9]:
%%R
# The significantly differentially expressed genes are the ones found in the upper-left and upper-right corners.
# Add a column to the data frame to specify if they are UP- or DOWN- regulated (log2FoldChange respectively positive or negative)

# add a column of NAs
volcano_data$diffexpressed <- "NO"
# if log2Foldchange > 0.6 and pvalue < 0.05, set as "UP" 
volcano_data$diffexpressed[volcano_data$logFC > 0.6 & volcano_data$P.Value < 0.05] <- "UP"
# if log2Foldchange < -0.6 and pvalue < 0.05, set as "DOWN"
volcano_data$diffexpressed[volcano_data$logFC < -0.6 & volcano_data$P.Value < 0.05] <- "DOWN"


In [10]:
%%R
head(volcano_data)

    LVs      logFC     adj_Pval negLogadj_Pval diffexpressed
1  LV37 -5.8620505 3.330138e-06       5.477538            NO
2  LV43  2.7419379 2.468918e-04       3.607493            NO
3  LV34  3.4855795 2.468918e-04       3.607493            NO
4   LV6 -6.1226084 2.653734e-04       3.576143            NO
5  LV83  1.8089640 1.841066e-03       2.734931            NO
6 LV100  0.9929464 2.852902e-03       2.544713            NO


In [11]:
%%R
library(EnhancedVolcano)
p1=EnhancedVolcano(volcano_data,
lab = volcano_data$LVs,
x = 'logFC',
y = 'adj_Pval',
title = 'D21/T21-BMI',
xlab = 'Log2 FC of LVs',
ylab = '-Log10 adj-P-value',
pCutoff = 0.05,
pCutoffCol = 'adj_Pval',
                
FCcutoff = 2,
pointSize = 3.0,
labSize = 6.0,
subtitle = 'Differential LVs',
legendLabels = c("NS", expression(Log[2] ~ FC), "adj-P", expression("adj-P-value" ~ and~ log[2] ~ FC)),)



ggsave("./output/volcano_plot_plier_LVS_d21_t21_BMI.pdf", plot = p1, width = 6, height = 6, dpi = 300, units = "in",bg="white", device = "pdf")


R[write to console]: Loading required package: ggplot2

R[write to console]: Loading required package: ggrepel

